In [1]:
import pandas as pd
import numpy as np

data = pd.read_feather('tavbase/Sample.feather')

In [2]:
def zscore(df, grupo, var, score):
    media = df[var].mean()
    dpadr = df[var].std()
    gr = df.groupby(grupo)[var]
    dc = gr.describe()
    dc = dc.reset_index()
    dc[score] = dc['mean'].apply(lambda x: (x - media) / dpadr)
    dcz = dc[[grupo, score]]
    df = df.merge(dcz, on=grupo, how='left')
    
    return df

In [3]:
## Classificacao zcore
variaveis_dimensoes = ['Customer ID', 'Segment', 'Region', 'Category']

for dimensao in variaveis_dimensoes:
    medidas = ['Sales', 'Profit', 'Quantity', 'Discount', 'Delivery', 'Price']
    grupo   = data.groupby(dimensao)[medidas].mean().reset_index()
    
    for medida in medidas: 
        grupo = zscore(grupo, dimensao, medida, 'z'+medida)
    
    zmedidas = ['z'+med for med in medidas]
    
    zgrupo = grupo[zmedidas]
    grupo['score'] = zgrupo.sum(axis=1)
    media_score    = grupo['score'].mean()
    dpadr_score    = grupo['score'].std()
    
    grupo['classe'] = grupo['score'].apply(lambda x : int((x - media_score) / dpadr_score) + 3)
    grupo['rank']  = grupo['score'].rank(ascending=False)
    grupo['lucro'] = grupo['Profit'].apply(lambda x : 0 if x < 0 else 1)

    grupo.to_feather('tavbase/classificacaoz_'+dimensao+'.feather')